In [4]:
import backbones_unet
from backbones_unet.model.unet import Unet
from backbones_unet.model.losses import DiceLoss,focal_loss,FocalLoss
from backbones_unet.utils.trainer import Trainer


from data_configs.penn_image_dataset import PennImageDataU
import utils.eval_tools as evl

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler
from sklearn.model_selection import RepeatedKFold
from torchmetrics.classification import BinaryConfusionMatrix
import matplotlib.pyplot as plt
import numpy as np

import os

from os import path as osp
import deepdish as dd
from attrdict import AttrDict

from tqdm import tqdm

# Setup and name experiment

In [10]:
k_inds = range(39)
exp_name = 'model_comparison_allk_crossVal_encoderFreeze0'
exp_name_short = 'allk_encoderfreeze'
MODELS = ['convnext_tiny_hnf','convnext_pico_ols','convnext_nano_ols','convnext_nano','convnext_femto','convnext_femto_ols']
config = AttrDict({'cross_validate':True,
                       'rot_augment':True,
                       'train_split':0.8,
                       'n_repeats':1,
                       'split_seed':3,
                       'input_channels':39,
                       'batch_size':10})

## Run experiment: cross-validate models

In [11]:
btch = config.batch_size
dat = PennImageDataU(rot_augment = config.rot_augment)
expdir = osp.join('model_experiments',exp_name)

kf = RepeatedKFold(n_splits =int(1/(1-config.train_split)), n_repeats = config.n_repeats, random_state = config.split_seed)

for i, (train_inds, test_inds) in enumerate(kf.split(range(len(dat)))):

    train_data = PennImageDataU(data_inds = train_inds,k_inds = k_inds,rot_augment = config.rot_augment)
    train_loader = DataLoader(train_data,batch_size = btch)
    test_data = PennImageDataU(data_inds = test_inds,k_inds = k_inds,rot_augment = config.rot_augment)
    test_loader = DataLoader(test_data,batch_size = btch)

    # TRAIN MODELS AND SAVE TO FILE
    for name in MODELS:
        print("Testing model",name)
        
        model = Unet(
        backbone=name, # backbone network name
        in_channels=len(k_inds),            # input channels (1 for gray-scale images, 3 for RGB, etc.)
        num_classes=2, # output channels (number of classes in your dataset)
        encoder_freeze = True,
        non_trainable_layers = (0,)
        )
        
        params = [p for p in model.parameters() if p.requires_grad]
        # print("number of params",len(params))
        optimizer = torch.optim.AdamW(params, 1e-4) 
    
        all_label_vals = torch.cat([torch.flatten(x) for x in train_data.labels])
        n_labels = len(all_label_vals)
        num_pos = torch.sum(all_label_vals).item()
        num_neg = n_labels - num_pos
        class_weights = [1/num_neg * (n_labels/2), (1/num_pos) * (n_labels/2)]
        
        trainer = Trainer(
            model,                    # UNet model with pretrained backbone
            criterion=FocalLoss(weights = class_weights),     # loss function for model convergence
            optimizer=optimizer,      # optimizer for regularization
            epochs=50,         # number of epochs for model training
            device = 'mps'
        )

        trainer.fit(train_loader, test_loader)
    
        loss_dict = AttrDict({'train_loss':trainer.train_losses_,'test_loss':trainer.val_losses_})
    
        loss_fname = osp.join('model_experiments',exp_name,name+ '_loss_fold{}.h5'.format(i+1))
    
        if not os.path.exists(expdir):
            os.makedirs(expdir)
            print("Folder created:", expdir)
    
        dd.io.save(loss_fname,loss_dict)
    
        model_state = trainer.model.state_dict()
    
        torch.save(model_state,osp.join('model_experiments',exp_name,name + '_wts_fold{}.pth'.format(i+1)))



Testing model convnext_tiny_hnf


Traning Model on 50 epochs:   0%|                        | 0/50 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ? training-batch/s]

  0%|          | 0/9 [00:00<?, ? validating-batch/s]

Traning Model on 50 epochs:   2%|▎               | 1/50 [00:06<05:14,  6.42s/it]

...

  0%|          | 0/9 [00:00<?, ? validating-batch/s]

Traning Model on 50 epochs: 100%|███████████████| 50/50 [03:26<00:00,  4.13s/it]


Working...
Loading 'pretrained_unet' from notebook_models/pretrained_unets/pretrained_unet.py
Loading 'penn_images' from notebook_models/Conv2D/penn_images.py


100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 21.01it/s]


FileNotFoundError: ``/Users/gorg/segmentation/model_experiments/pretrained_model_comparison_allk_crossVal/convnext_tiny_hnf_loss_fold2.h5`` does not exist

### Compute classifier metrics for model experiment

In [12]:
# Compute metrics
for model_name in MODELS:
    config['backbone'] = model_name
    model_config = osp.join('notebook_models','pretrained_unets','pretrained_unet.py')
    data_config = osp.join('notebook_models','Conv2D','penn_images.py')
    
    info = evl.get_image_metrics(model_config,data_config,exp_name,config,disable_tqdm = False)

    save_dir = osp.join('notebook_models',model_name)
    if not os.path.exists(save_dir):
        # Create the directory
        os.makedirs(save_dir)
    save_file = osp.join(save_dir,exp_name_short+'_metrics.h5')
    dd.io.save(save_file,info)
    dd.io.save('encoderfreeze_config.h5',config)

Working...
Loading 'pretrained_unet' from notebook_models/pretrained_unets/pretrained_unet.py
Loading 'penn_images' from notebook_models/Conv2D/penn_images.py


100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 22.38it/s]


done.
Working...
Loading 'pretrained_unet' from notebook_models/pretrained_unets/pretrained_unet.py
Loading 'penn_images' from notebook_models/Conv2D/penn_images.py


100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 24.03it/s]


done.
Working...
Loading 'pretrained_unet' from notebook_models/pretrained_unets/pretrained_unet.py
Loading 'penn_images' from notebook_models/Conv2D/penn_images.py


...

done.
Working...
Loading 'pretrained_unet' from notebook_models/pretrained_unets/pretrained_unet.py
Loading 'penn_images' from notebook_models/Conv2D/penn_images.py


100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 22.93it/s]


done.
Working...
Loading 'pretrained_unet' from notebook_models/pretrained_unets/pretrained_unet.py
Loading 'penn_images' from notebook_models/Conv2D/penn_images.py


100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 22.23it/s]


done.
